In [1]:
import os
import shutil
import time
import concurrent.futures
import requests
import pandas as pd
import re

In [2]:
start_time = time.time()
print(f"start time: {time.strftime('%x %X')}")

start time: 02/22/20 23:38:09


In [3]:
dataset = pd.read_excel("asins.xlsx")

In [11]:
dataset

,asin
0,B01MSUOG4A
1,B07FTG9PBK
2,B0041KVW2K
3,B000C9SNFI
4,B00006B8G8
5,B000PDNKFE
6,B00284B5HI
7,B001B09AHW
8,B000HCR89M
9,B001U6CXCG


In [4]:
url_part = """https://www.amazon.com/gp/customer-reviews/widgets/average-customer-review/popover/ref=dpx_acr_pop_?contextId=dpx&asin="""

In [5]:
error_list = []
result = []
c1 = re.compile(r"[1-5][.]?[0-9]? out of 5")
c2 = re.compile(r"[0-9,]+  customer ratings")


def reviews_ratings(asin):
    url =  url_part + asin
    f = requests.get(url)
    if f.status_code == 200:
        ratings = c1.findall(str(f.content))[0]
        reviews = c2.findall(str(f.content))[0]
        result.append([asin, ratings, reviews])
    else:
        error_list.append(str(asin))

In [15]:
for i in range(len(dataset)):
    reviews_ratings(dataset.iloc[i])
    

asin    https://www.amazon.com/gp/customer-reviews/wid...
Name: 0, dtype: object


InvalidSchema: No connection adapters were found for 'asin    https://www.amazon.com/gp/customer-reviews/wid...
Name: 0, dtype: object'

In [9]:
result

[]

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(reviews_ratings, dataset.iloc[0:])

In [ ]:
error_list

In [ ]:
#converting result and error list to dataframe
result = pd.DataFrame(result)
result.columns = ['asin', 'ratings', 'reviews']

error_list = pd.DataFrame(error_list)
print(f"\n\nFor below asins we got error: \n{error_list}" ) 

In [ ]:
error_list.to_csv("Error_File.csv", index = False)
result.to_csv("review_ratings.csv", index = False)

In [ ]:
end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f"end time: {time.strftime('%x %X')}")
print(f"Approx. time taken to get all the values: {round(minutes)} minutes {round(seconds)} seconds")